# Startup for Germany
This code is to be included at the beginning of `calib_GER` and `simu_GER`.

In [1]:
t0 = time() ## start chrono
dir_module = joinpath(pwd(),"src")
dir_graph = joinpath(pwd(),"../../graphs")
    
########### main module
using NBInclude
@nbinclude(joinpath(dir_module,"AgeDisc.ipynb"))
using Main.AgeDisc

########### other packages
using QuadGK, DataFrames, PyPlot

### Some variables can be predefined before startup.jl : _PRECOMPILE_, _LC_
## _PRECOMPILE_ is whether to run a few functions so that they are already compiled
if !@isdefined(_PRECOMPILE_)
    _PRECOMPILE_ = true
end

if !@isdefined(_LC_)
    _LC_ = false
end

false

# Moments

In [2]:
data_ger_UI = DataFrame([
(40+1/6, 6.73),
(40+2/6, 6.71),
(40+3/6, 6.74),
(40+4/6, 6.76),
(40+5/6, 6.75),
(41., 6.77),
(41+1/6, 6.82),
(41+2/6, 6.83),
(41+3/6, 6.85),
(41+4/6, 6.81),
(41+5/6, 6.8),
(42+1/6, 8.57),
(42+2/6, 8.76),
(42+3/6, 8.74),
(42+4/6, 8.72),
(42+5/6, 8.8),
(43., 8.79),
(43+1/6, 8.94),
(43+2/6, 8.89),
(43+3/6, 8.98),
(43+4/6, 8.91),
(43+5/6, 8.99)], [:age, :rate])
data_ger_UI.age *= 12

data_ger_NE = DataFrame([
(40+1/6, 15.52),
(40+2/6, 15.46),
(40+3/6, 15.57),
(40+4/6, 15.5),
(40+5/6, 15.46),
(41., 15.54),
(41+1/6, 15.55),
(41+2/6, 15.69),
(41+3/6, 15.6),
(41+4/6, 15.73),
(41+5/6, 15.54),
(42+1/6, 16.5),
(42+2/6, 16.51),
(42+3/6, 16.41),
(42+4/6, 16.37),
(42+5/6, 16.44),
(43., 16.52),
(43+1/6, 16.65),
(43+2/6, 16.61),
(43+3/6, 16.68),
(43+4/6, 16.48),
(43+5/6, 16.68)], [:age, :rate])    
data_ger_NE.age *= 12


moments_target, Age_vec = let
    Age_bds = (41, 43)  .*12 ## window to match the moments
   # myselec_step1 = [i % 3 == 0 for i in 1:length(mydata.age)]
    myselec = (Age_bds[1] .<= data_ger_NE.age .<= Age_bds[2]) # .& myselec_step1 
    myselec = Age_bds[1] .<= data_ger_NE.age .<= Age_bds[2]
    (data_ger_NE.rate[myselec], data_ger_NE.age[myselec])
end

moments_target_UI = let
    Age_bds = (41, 43)  .*12 ## window to match the moments
   # myselec_step1 = [i % 3 == 0 for i in 1:length(mydata_UI.age)]
    myselec = (Age_bds[1] .<= data_ger_UI.age .<= Age_bds[2]) # .& myselec_step1 
   mom = (data_ger_UI.rate[myselec], data_ger_NE.rate[myselec], (892.9+28/2)/365*12)
   mom
end

function moments_germany(V::Surplus, C::Country2A;
             a_range=Age_vec)
    ## ex: for age a in data, we compute the average between D(a-1/6), D(a) and D(a+1/6) using 
    Y_vec = zeros(2+length(a_range))
    toadd = expectedD(a_range[1]-2,V,C)
    Y_vec[1] += toadd/6 
    toadd = expectedD(a_range[1],V,C)
    Y_vec[1] += toadd*2/3
    Y_vec[2] += toadd/6
    for (i,a) in enumerate(a_range[2:end])
        j = i+1 ## j from 2 to end
        if a+2==C.A
            toadd = expectedD(a,V,C)
            Y_vec[j-1] += toadd/6 
            Y_vec[j] += toadd *2/3
            toaddA = expectedD(C.A-0.001,V,C)
            Y_vec[j] += toaddA /6
        elseif a-2==C.A
            toaddA = expectedD(C.A,V,C)
            Y_vec[j] += toaddA /6
            toadd = expectedD(a,V,C)
            Y_vec[j] += toadd *2/3
            Y_vec[j+1] += toadd/6 
        else
            toadd = expectedD(a,V,C)
            Y_vec[j-1] += toadd/6 
            Y_vec[j] += toadd *2/3
            Y_vec[j+1] += toadd /6
        end
    end
    toadd = expectedD(a_range[end]+2,V,C)
    Y_vec[length(a_range)] += toadd/6
    Y_vec[1:length(a_range)]
end

#=   
function moments_germany(V::Surplus, C::Country2A)
    Y = [expectedD(a,V,C) for a in Age_vec]
    return Y
end=#


function moments_germany_UInoq(V::Surplus, C::Country2A;
             a_range=Age_vec)
    ## ex: for age a in data, we compute the average between D(a-1.5) and D(a+1.5)
    Y1_vec = zeros(2+length(a_range))
    Y2_vec = zeros(2+length(a_range))
    
    toadd1 = expectedB(a_range[1]-2,V,C)
    Y1_vec[1] += toadd1/6 
    toadd2 = expectedD(a_range[1]-2,V,C)
    Y2_vec[1] += toadd2/6 
    
    toadd1 = expectedB(a_range[1],V,C)
    Y1_vec[1] += toadd1*2/3
    Y1_vec[2] += toadd1/6
    toadd2 = expectedD(a_range[1],V,C)
    Y2_vec[1] += toadd2*2/3
    Y2_vec[2] += toadd2/6
    
    for (i,a) in enumerate(a_range[2:end])
        j = i+1 ## j from 2 to end
        if a+2==C.A
            toadd1 = expectedB(a,V,C)
            Y1_vec[j-1] += toadd1/6 
            Y1_vec[j] += toadd1 *2/3
            toadd2 = expectedD(a,V,C)
            Y2_vec[j-1] += toadd2/6 
            Y2_vec[j] += toadd2 *2/3
            
            toaddA1 = expectedB(C.A-0.001,V,C)
            Y1_vec[j] += toaddA1 /6
            toaddA2 = expectedD(C.A-0.001,V,C)
            Y2_vec[j] += toaddA2 /6
            
        elseif a-2==C.A
            toaddA1 = expectedB(C.A,V,C)
            Y1_vec[j] += toaddA1 /6
            toaddA2 = expectedD(C.A,V,C)
            Y2_vec[j] += toaddA2 /6
            
            toadd1 = expectedB(a,V,C)
            Y1_vec[j] += toadd1 *2/3
            Y1_vec[j+1] += toadd1/6 
            toadd2 = expectedD(a,V,C)
            Y2_vec[j] += toadd2 *2/3
            Y2_vec[j+1] += toadd2/6 
        else
            toadd1 = expectedB(a,V,C)
            Y1_vec[j-1] += toadd1/6 
            Y1_vec[j] += toadd1 *2/3
            Y1_vec[j+1] += toadd1 /6
            toadd2 = expectedD(a,V,C)
            Y2_vec[j-1] += toadd2/6 
            Y2_vec[j] += toadd2 *2/3
            Y2_vec[j+1] += toadd2 /6
        end
    end
    toadd1 = expectedB(a_range[end]+2,V,C)
    Y1_vec[length(a_range)] += toadd1/6
    toadd2 = expectedD(a_range[end]+2,V,C)
    Y2_vec[length(a_range)] += toadd2/6
    return (Y1_vec[1:length(a_range)], Y2_vec[1:length(a_range)])
end

#=
function moments_germany_UInoq(V::Surplus, C::Country2A;
             a_range=Age_vec)
    Y1 = [expectedB(a,V,C) for a in a_range]
    Y2 = [expectedD(a,V,C) for a in a_range]
    return (Y1,Y2)
end
=#



function moments_germany_UI(V::Surplus, C::Country2A;
             a_range=Age_vec)
    #Y1 = [expectedB(a,V,C) for a in a_range]
    #Y2 = [expectedD(a,V,C) for a in a_range]
    (Y1,Y2) = moments_germany_UInoq(V, C, a_range=a_range)
    momav = average_U5(V, C, Nind=100000) 
    return (Y1,Y2,momav)
end


moments_germany_UI (generic function with 1 method)

# Parameters

In [3]:
if !_LC_
    C0 = let
        Country2(
            A = 42. * 12, Amin = 38. *12, Amax = 44. *12,
            r = 0.0048, # 5 % annual interest rate, time unit = month
            α = NaN, m = NaN, q = NaN, λ0 = NaN, λ1 = NaN,

            bUI = 0.68,   # replacement rate for individual with children 68 (without children is 63) , SVB2012a
            b0 = 0.35, # 35% social assistance for married individuals, SVB2012a
            B1=NaN, B2=NaN, B1dif=NaN
        )    
    end

    function update_Country2(;α=C0.α, m=C0.m, q=C0.q, λ1=C0.λ1, λ0=C0.λ0)
        (B1, B1dif, B2) = Baux(α=α, m=m) 
        Country2(A=C0.A, Amin=C0.Amin, Amax=C0.Amax,
                        r=C0.r, q=q, m=m, λ1=λ1, λ0=λ0, α=α,
                        bUI=C0.bUI, b0=C0.b0,
                        B1=B1, B1dif=B1dif, B2=B2)
    end

    if _PRECOMPILE_
        let 
            germany = update_Country2(α=0.81, m=0.11, q= 0.026, λmin= 0.065, λmax= 0.065 + 0.027)  
            a=41.8*12
            V = find_Surplus(germany,reltol=1e-3, findΨ=false) 
            expectedB(a, V, germany)
            expectedD(a, V, germany)
            average_U5(V, germany, Nind=10) 
        end  
    end

else
    C0 = let
        Country2A(
            A = 42. * 12, Amin = 38. *12, Amax = 44. *12, Aret=  65. *12,
            r = 0.0048, # 5 % annual interest rate, time unit = month
            α = NaN, m = NaN, q0 = NaN, qa=NaN, λ0 = NaN, λ1 = NaN,
            χ = 0.7, #source Trenkle (2023)
            bUI = 0.68,   # replacement rate for individual with children 68 (without children is 63) , SVB2012a
            b0 = 0.35, # 35% social assistance for married individuals, SVB2012a
            B1=NaN, B2=NaN, B1dif=NaN
        )    
    end

    ## 2) function to update Country
    function update_Country2(;α=C0.α, m=C0.m, q0=C0.q0, qa=C0.qa, λ1=C0.λ1, λ0=C0.λ0, χ=C0.χ, Aret=C0.Aret)
        (B1, B1dif, B2) = Baux(α=α, m=m) 
        Country2A(A=C0.A, Amin=C0.Amin, Amax=C0.Amax, Aret=Aret,
                        r=C0.r, q0=q0, qa=qa, m=m, λ1=λ1, λ0=λ0, α=α, χ=χ,
                        bUI=C0.bUI, b0=C0.b0,
                        B1=B1, B1dif=B1dif, B2=B2)
    end

    if _PRECOMPILE_
        let 
            germany = update_Country2(α=0.81, m=0.11, q0= 0.026, qa=0., λ1= 0.065, λ0= 0.065 + 0.027)  
            a=41.8*12
            V = find_Surplus(germany,reltol=1e-3, findΨ=false) 
            expectedB(a, V, germany)
            expectedD(a, V, germany)
            average_U5(V, germany, Nind=10) 
        end  
    end
end

LoadError: MethodError: no method matching update_Country2(; α::Float64, m::Float64, q::Float64, λmin::Float64, λmax::Float64)

[0mClosest candidates are:
[0m  update_Country2(; α, m, q, λ1, λ0)[91m got unsupported keyword arguments "λmin", "λmax"[39m
[0m[90m   @[39m [35mMain[39m [90m[4mIn[3]:14[24m[39m


In [4]:
println("C0 is $C0")
t1 = time()
nsec = (t1-t0)
println("Startup run in $nsec secs.")

C0 is Country2(504.0, 456.0, 528.0, 0.0048, NaN, NaN, NaN, NaN, NaN, 0.68, 0.35, NaN, NaN, NaN)
Startup run in 31.503999948501587 secs.
